# TODO
* Deck
    * What is Data Science?
    * Why do we want to predict/understand things?
    * What forms these things can take.
* Here
    * Intro to our example & linear regression
    * Train a basic model
    * Evaluate model & discuss other evaluation metrics
    * Talk about deployment, potentially have one deployed they can hit with an API

# What are we doing here?
This is the Infinityworks Data Science 101 workshop. We will be training a model to predict...*drum roll*...The weight of various star wars characters based on their other vital statistics.

Don't say we didn't try to make this exciting!

## Wait, what is this sourcery? Am in an IDE here or what?
This is a Jupyter Notebook. It's a standard bit of kit in Data Science circles. It's shite for proper code dev but there's few better tools for doing investigative coding and mixing it with notes and charts. If you want to learn a bit more then there's a link [here]().

For now, just enjoy the ride and have a play. If, for example, you click on this text you can edit it as Markdown.

TODO: Add the link to my notebook 101.

### First things first, we need some libraries

In [5]:
import pandas as pd     # Standard Python data handling library. Lets you store tables (often called `DataFrames`)
import seaborn as sns   # Plotting library, there's many to choose from but this is a stalwart

# Sklearn is the standard Python stats tooling. It's...large and unwieldy but goes.
from sklearn.linear_model import LinearRegression
from sklearn.


### Now, some data

In [3]:
df = pd.read_csv("characters.csv")
df = df[~pd.isnull(df.mass)]
df.mass = df.mass.str.replace(",", "").astype("float")
df.height = df.height.astype("float")
df

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19BBY,male,Tatooine,Human
1,C-3PO,167.0,75.0,NaN,gold,yellow,112BBY,NaN,Tatooine,Droid
2,R2-D2,96.0,32.0,NaN,"white, blue",red,33BBY,NaN,Naboo,Droid
3,Darth Vader,202.0,136.0,none,white,yellow,41.9BBY,male,Tatooine,Human
4,Leia Organa,150.0,49.0,brown,light,brown,19BBY,female,Alderaan,Human
5,Owen Lars,178.0,120.0,"brown, grey",light,blue,52BBY,male,Tatooine,Human
6,Beru Whitesun lars,165.0,75.0,brown,light,blue,47BBY,female,Tatooine,Human
7,R5-D4,97.0,32.0,NaN,"white, red",red,NaN,NaN,Tatooine,Droid
8,Biggs Darklighter,183.0,84.0,black,light,brown,24BBY,male,Tatooine,Human
9,Obi-Wan Kenobi,182.0,77.0,"auburn, white",fair,blue-gray,57BBY,male,Stewjon,Human


## Job one is *always* to look at your data first
TODO: Some plots

## Now, let's model!

In [ ]:
train_mass, train_height = sk

In [14]:
mod = LinearRegression()
mod.fit(df.mass.to_numpy().reshape(-1,1), df.height)

LinearRegression()

# Great, is it any good?


## How do we know if it's any good?
We do NOT use p-values. P-values are a plauge on the academic sciences and (in general) have even less value in business. If you don't know what these are that's fine, you have an advantage here. If you do, park that.

Remember, in business we want to: Increase profit, reduce risk or increase efficiencies. All of these can be done whether your p-value is significant or not!

## Is our model any good?